In [5]:
# *Spam Email Detection	*

# Goal: Build a model to classify emails as spam or non-spam.
# Steps:

# Frontend: Streamlit.
# Dataset: SMS Spam Collection Dataset
# Model: Use Logistic Regression, Naive Bayes, or LSTM for text classification.
# Evaluation: Precision, recall, and F1-score.

# https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset 

In [2]:
pip install pandas scikit-learn tensorflow numpy


  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
  Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl (7.5 kB)
  Using cached numpy-2.2.3-cp310-cp310-win_amd64.whl (12.9 MB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl (41.2 MB)
  Using cached ml_dtypes-0.4.1-cp310-cp310-win_amd64.whl (126 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl (434 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached grpcio-1.70.0-cp310-cp310-win_amd64.whl (4.3 MB)
  Using cached termcolor-2.5.0-py3-none-any.whl (7.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.wh

You should consider upgrading via the 'c:\Users\FARAZ TRADERS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: keras in c:\users\faraz traders\appdata\local\programs\python\python310\lib\site-packages (3.8.0)



You should consider upgrading via the 'c:\Users\FARAZ TRADERS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install --upgrade tensorflow keras

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\FARAZ TRADERS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

2.18.0
3.8.0


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional
from tensorflow.keras.optimizers import Adam

# Load dataset
df = pd.read_csv('spam.csv', usecols=[0, 1], names=['label', 'message'], header=0, encoding='latin1')

# Preprocessing: Convert labels to binary values (0 for ham, 1 for spam)
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Machine Learning Models: Using TF-IDF Vectorization
# Create TF-IDF Vectorizer to convert text data into numerical form
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=3000)

# Fit and transform the training data, transform the testing data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Standardize for logistic regression
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_tfidf)
X_test_scaled = scaler.transform(X_test_tfidf)

# Train ML models
models = {
    'Naïve Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear', C=1.0),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Logistic Regression': LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)
}
 
# Train and evaluate each machine learning model
results = {}
for name, model in models.items():
    if name == 'Naive Bayes':  # Naive Bayes works directly with sparse matrices
        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
    else:  # Other models require dense arrays
        model.fit(X_train_tfidf.toarray(), y_train)
        y_pred = model.predict(X_test_tfidf.toarray())

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    results[name] = {'accuracy': accuracy, 'report': report}

# Deep Learning Model: Using LSTM for Sequence Classification
# Tokenize messages and pad sequences for deep learning models
max_features = 5000  # Maximum number of words to consider
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['message'].values)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train.values)
X_test_seq = tokenizer.texts_to_sequences(X_test.values)

# Pad sequences to ensure uniform input size
max_len = 20  # Maximum length of each sequence
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Build an LSTM-based neural network model
model = Sequential([
        Embedding(max_features, 256, input_length=max_len),
        SpatialDropout1D(0.3),
        Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)),
        Dense(1, activation='sigmoid')
    ]) # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test), verbose=2)

# Evaluate the LSTM model
y_pred_dl = (model.predict(X_test_pad) > 0.5).astype("int32")
dl_accuracy = accuracy_score(y_test, y_pred_dl)
dl_report = classification_report(y_test, y_pred_dl, zero_division=0)
results['Deep Learning (LSTM)'] = {'accuracy': dl_accuracy, 'report': dl_report}

# Print results for all models
print("\n=== SPAM DETECTION MODEL RESULTS ===\n")
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print("Classification Report:\n", result['report'])
    print("=" * 50)

c:\Users\FARAZ TRADERS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
140/140 - 20s - 143ms/step - accuracy: 0.9569 - loss: 0.1261 - val_accuracy: 0.9848 - val_loss: 0.0588
Epoch 2/5
140/140 - 12s - 83ms/step - accuracy: 0.9915 - loss: 0.0301 - val_accuracy: 0.9812 - val_loss: 0.0783
Epoch 3/5
140/140 - 10s - 74ms/step - accuracy: 0.9953 - loss: 0.0177 - val_accuracy: 0.9821 - val_loss: 0.0692
Epoch 4/5
140/140 - 11s - 76ms/step - accuracy: 0.9975 - loss: 0.0087 - val_accuracy: 0.9812 - val_loss: 0.0803
Epoch 5/5
140/140 - 10s - 73ms/step - accuracy: 0.9993 - loss: 0.0033 - val_accuracy: 0.9821 - val_loss: 0.0822
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step

=== SPAM DETECTION MODEL RESULTS ===

Model: Naïve Bayes
Accuracy: 0.9776
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.83      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98     